<a href="https://colab.research.google.com/github/ez615/ResNet-Practice/blob/main/ResNet_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resnet 논문 구현**

출처: https://github.com/CryptoSalamander/pytorch_paper_implementation/tree/master/resnet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class BasicBlock(nn.Module):
  mul = 1
  def __init__(self, in_planes, out_planes, stride=1):
    super(BasicBlock, self).__init__()

    self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_planes)

    self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_planes)

    self.shortcut = nn.Sequential()

    if stride != 1:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(out_planes)
      )

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out += self.shortcut(x)
    out = F.relu(out)

    return out

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=7, stride=2, padding=3)
    self.bn1 = nn.BatchNorm2d(self.in_planes)
    self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self.make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self.make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self.make_layer(block, 512, num_blocks[3], stride=2)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    self.linear = nn.Linear(512 * block.mul, num_classes)

  def make_layer(self, block, out_planes, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []

    for i in range(num_blocks):
      layers.append(block(self.in_planes, out_planes, strides[i]))
      self.in_planes = block.mul * out_planes

      return nn.Sequential(*layers)

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.maxpool1(out)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.avgpool(out)
    out = torch.flatten(out, 1)
    out = self.linear(out)

    return out

In [ ]:
def Resnet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

def Resnet34():
  return ResNet(BasicBlock, [3, 4, 6, 3])

**ResNet 학습 구현**

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import os
import torchvision.models as models

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def lr_scheduler(optimizer, epoch):
    lr = 0.1
    if epoch >= 50:
        lr /= 10
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
         
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
device = 'cuda'
model = Resnet18()
model.apply(init_weights)
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if sys.path[0] == '':


In [ ]:
learning_rate = 0.1
num_epoch = 100
model_name = 'resnet18.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

# train_loss = 0
# valid_loss = 0
# correct = 0
# total_cnt = 0
best_acc = 0

In [ ]:
#Train
for epoch in range(num_epoch):
  print(f"====== {epoch + 1} epoch of {num_epoch} ======")

  model.train()
  lr_scheduler(optimizer, epoch)
  train_loss = 0
  valid_loss = 0
  correct = 0
  total_cnt = 0

  for step, batch in enumerate(train_loader):
    batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
    optimizer.zero_grad()

    logits = model(batch[0])
    loss = loss_fn(logits, batch[1])
    loss.backward()

    optimizer.step()
    train_loss += loss.item()
    _, predict = logits.max(1)

    total_cnt += batch[1].size(0)
    correct += predict.eq(batch[1]).sum().item()

    if step % 100 == 0 and step != 0:
      print(f"\n====== {step} Step of {len(train_loader)} ======")
      print(f"Train Acc : {correct / total_cnt}")
      print(f"Train Loss : {loss.item() / batch[1].size(0)}")

  correct = 0
  total_cnt = 0

  with torch.no_grad():
    model.eval()
    for step, batch in enumerate(test_loader):
      batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
      total_cnt += batch[1].size(0)
      logits = model(batch[0])
      valid_loss += loss_fn(logits, batch[1])
      _, predict = logits.max(1)
      correct += predict.eq(batch[1]).sum().item()
    valid_acc = correct / total_cnt
    print(f"\nValid Acc : { valid_acc }")    
    print(f"Valid Loss : { valid_loss / total_cnt }")

    if(valid_acc > best_acc):
      best_acc = valid_acc
      torch.save(model, model_name)
      print("Model Saved!")

====== 1 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.2848855198019802
Train Loss : 0.006884220987558365

Valid Acc : 0.4276
Valid Loss : 0.006293554324656725
Model Saved!
====== 2 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.42918471534653463
Train Loss : 0.005532798822969198

Valid Acc : 0.5079
Valid Loss : 0.005372197832912207
Model Saved!
====== 3 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.49632580445544555
Train Loss : 0.005517440848052502

Valid Acc : 0.5081
Valid Loss : 0.005502105690538883
Model Saved!
====== 4 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.5595606435643564
Train Loss : 0.004229879006743431

Valid Acc : 0.4201
Valid Loss : 0.00694545591250062
====== 5 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.5930151608910891
Train Loss : 0.004507437814027071

Valid Acc : 0.6089
Valid Loss : 0.004408051259815693
Model Saved!
====== 6 epoch of 100 ======

====== 100 Step of 196